In [16]:
from nltk.corpus import wordnet,stopwords
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer
import string
import nltk
from sklearn.model_selection import GridSearchCV,cross_val_score
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [17]:
training_data = pd.read_csv('train.csv',encoding='latin-1')
# test_text = pd.read_csv('test.csv')

In [18]:
training_data

,ItemID,Sentiment,SentimentText
0,1,0,is so sad for my APL frie...
1,2,0,I missed the New Moon trail...
2,3,1,omg its already 7:30 :O
3,4,0,.. Omgaga. Im sooo im gunna CRy. I'...
4,5,0,i think mi bf is cheating on me!!! ...
...,...,...,...
99984,99996,0,@Cupcake seems like a repeating problem hop...
99985,99997,1,@cupcake__ arrrr we both replied to each other...
99986,99998,0,@CuPcAkE_2120 ya i thought so
99987,99999,1,@Cupcake_Dollie Yes. Yes. I'm glad you had mor...


In [19]:
import re

In [21]:
ans = []
def camelCases(string):
    splitted = re.sub('([A-Z][a-z]+)', r' \1', re.sub('([A-Z]+)', r' \1', string)).split()
#     print(splitted)
    string = ' '.join([str(elem) for elem in splitted])
    listToStr = string.translate ({ord(c): " " for c in "!@#$%^&*()[]{};:,./<>?\|`~-=_+0123456789"})
    ans.append(str(listToStr).lower())

In [22]:
def clean(x):
    x = x.lower()
    x = re.sub(r'(https?:\/\/)(\s)*(www\.)?(\s)*((\w|\s)+\.)*([\w\-\s]+\/)*([\w\-]+)((\?)?[\w\s]*=\s*[\w\%&]*)*', '', x, flags=re.MULTILINE)
#     print(x)
    x = re.sub(r'[^a-zA-Z]', ' ', x, flags=re.MULTILINE)    
#     print(x)
    return x

In [23]:
training_data['SentimentText'] = training_data['SentimentText'].str.encode("ascii","ignore")
training_data['SentimentText'] = training_data['SentimentText'].apply(lambda x: x.decode("utf-8"))

In [24]:
training_data['SentimentText'] = training_data['SentimentText'].apply(lambda x: clean(x))

In [26]:
for i in range(training_data.shape[0]):
    camelCases(training_data['SentimentText'].iloc[i])

In [27]:
training_data['SentimentText'] = training_data['SentimentText'].apply(lambda x: clean(x))

In [31]:
from nltk.corpus import stopwords
list_of_stopwords = list(stopwords.words('english')) + list(string.punctuation)
lemmatizer = WordNetLemmatizer()

In [32]:
clean_tweet = []
for i in range(training_data.shape[0]):
    text_message = training_data['SentimentText'].values[i]
    tokenized_words = word_tokenize(text_message)
    clean_text = ''
    for word in tokenized_words:
        if not word in list_of_stopwords and word not in list("abcdefghijklmnopqrstuvwxyz"):
            clean_text += lemmatizer.lemmatize(word) + ' '
            
    clean_tweet.append(clean_text)
    
len(clean_tweet)

99989

In [33]:
training_data['SentimentText'] = clean_tweet

In [34]:
training_data.head()

0                                          sad apl friend 
1                                 missed new moon trailer 
2                                             omg already 
3        omgaga im sooo im gunna cry dentist since supo...
4                                    think mi bf cheating 
                               ...                        
99984    cupcake seems like repeating problem hope able...
99985    cupcake arrrr replied different tweet time see...
99986                                  cupcake ya thought 
99987                     cupcake dollie yes yes glad fun 
99988                              cupcake kayla haha yes 
Name: SentimentText, Length: 99989, dtype: object

In [43]:
cv = CountVectorizer()
sparse_matrix_tweet = cv.fit_transform(training_data['SentimentText'])

In [44]:
sparse_matrix_tweet.shape

(99989, 90556)

In [45]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(sparse_matrix_tweet, training_data['Sentiment'])
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((74991, 90556), (24998, 90556), (74991,), (24998,))

In [46]:
from sklearn.naive_bayes import MultinomialNB
nv = MultinomialNB()
nv.fit(X_train, Y_train)

from sklearn.metrics import accuracy_score
y_pred = nv.predict(X_test)
accuracy_score(Y_test, y_pred)

0.740099207936635

In [47]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {}'.format(accuracy_score(Y_test, y_pred)))

Accuracy of logistic regression classifier on test set: 0.7476198095847668


C:\Users\iota\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [84]:
# from sklearn.naive_bayes import MultinomialNB
# model1 = MultinomialNB()
# cross_val_score(model1,X_train,Y_train).mean()

In [85]:
# from sklearn.linear_model import LogisticRegression
# model2 = LogisticRegression(max_iter=2000,multi_class='ovr')
# cross_val_score(model2,X_train,Y_train).mean()

In [86]:
# model2.fit(X_train,Y_train)

In [73]:
# model2.predict(X_test)

In [74]:
# news = np.array(["i am bad","he is fine now","all is well"])
# news = cv.transform(news)
# model2.predict(news)
# # news

first channel : india today

In [75]:
from bs4 import BeautifulSoup
import requests

res = requests.get('https://www.indiatoday.in/')
soup = BeautifulSoup(res.text, 'lxml')

news_box = soup.find('ul', {'class': 'itg-listing'})
# all_news = news_box.find_all('a')
# print(all_news)
links = list()
for a in news_box.find_all('a', href=True):
    print( "Found the URL:", a['href'])
    print()
    links.append(a['href'])
    
# for news in all_news:
#     print(news.text)
#     print()

all_news = list()
for link in links:
    res = requests.get('https://www.indiatoday.in' + link)
    soup = BeautifulSoup(res.text, 'lxml')
    news_box = soup.find('div', {'class': 'description'})
    try :
        news_description = news_box.find_all('p')
        story = ""
        for news in news_description:
            story = story + news.text
        all_news.append(story)
    except AttributeError :
        continue
    
for news in all_news:
    print(news)
    print()
#     break    
    
    

Found the URL: /sports/ipl-2020/story/ipl-2020-pragyan-ojha-predicts-top-4-teams-qualify-for-playoffs-mi-dc-rcb-kxip-1736836-2020-10-31

Found the URL: /business/story/income-tax-return-filing-basic-exemption-limit-new-and-old-regime-1736821-2020-10-31

Found the URL: /coronavirus-outbreak/story/700-km-long-traffic-jam-people-paris-flee-france-ahead-coronavirus-lockdown-1736837-2020-10-31

Found the URL: /technology/news/story/pubg-mobile-app-removed-india-servers-now-shut-down-the-full-story-explained-in-5-points-1736832-2020-10-31

Found the URL: /elections/bihar-assembly-polls-2020/story/ec-branch-bjp-surprised-rjd-tejashwi-yadav-becomes-bihar-cm-shiv-sena-sanjay-raut-1736817-2020-10-31

Found the URL: /impact-feature/story/top-5-cloud-computing-courses-in-india-1736339-2020-10-29

Found the URL: /impact-feature/story/find-the-pc-that-s-just-right-for-you-1736485-2020-10-30

Found the URL: /impact-feature/story/winning-at-poker-is-like-first-love-you-just-can-t-stop-smiling-1736340-

In [76]:
news1 = np.array(all_news)
news1 = cv.transform(news1)
# nv.predict(news1)

array([0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0], dtype=int64)

In [77]:
logreg.predict(news1)

array([0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0], dtype=int64)

In [ ]:
#for india today out of 17, positive news are 5 and 12 are negatives.

second channel : times of india

In [78]:
res = requests.get('https://timesofindia.indiatimes.com/')
soup = BeautifulSoup(res.text, 'lxml')

news_box = soup.find('ul', {'class': 'list8'})
# all_news = news_box.find_all('a')
# print(all_news)
links = list()
for a in news_box.find_all('a', href=True):
    print( "Found the URL:", a['href'])
    print()
    links.append(a['href'])
    
# for news in all_news:
#     print(news.text)
#     print()

all_news = list()
for link in links:
    res = requests.get('https://timesofindia.indiatimes.com/' + link)
    soup = BeautifulSoup(res.text, 'lxml')
    news_box = soup.find('div', {'class': 'ga-headlines'})
    try:
        print(news_box.text)
        print()
        all_news.append(news_box.text)
    except AttributeError:
        continue
#     news_description = news_box.find_all('p')
#     story = ""
#     for news in news_description:
#         story = story + news.text
#     all_news.append(story)
    
    
for news in all_news:
    print(news)
    print()
#     break    
    

Found the URL: /india/covid-vaccination-drive-may-span-over-a-year-group-wise/articleshow/78961827.cms

Found the URL: /elections/assembly-elections/bihar/ec-gives-clean-chit-to-bjps-free-vaccine-promise-in-bihar-says-no-violation-of-model-code/articleshow/78966753.cms

Found the URL: /india/pulwama-terror-attack-truth-revealed-after-claims-in-pakistan-parliament-pm-modi/articleshow/78964354.cms

Found the URL: /india/pulwama-terror-attack-truth-revealed-after-claims-in-pakistan-parliament-pm-modi/articleshow/78964354.cms

Found the URL: /sports/cricket/ipl/top-stories/dc-vs-mi-live-score-updates-ipl-2020/articleshow/78966805.cms

Found the URL: /world/europe/third-person-under-arrest-france-in-connection-with-nice-attack/articleshow/78967208.cms

Found the URL: /world/us/us-presidential-elections/why-trump-will-almost-certainly-declare-himself-a-winner-on-nov-3/articleshow/78964344.cms

Found the URL: /world/us/us-presidential-elections/why-trump-will-almost-certainly-declare-himself-

NEW DELHI: Prime Minister Narendra Modi on Saturday said that the truth behind last year's Pulwama terror attack has been revealed after confession of a Pakistani minister in Parliament and slammed the opposition for raising questions in the aftermath of it. In a sensational admission, senior Pakistani minister Fawad Chaudhry Thursday admitted that his country was responsible for the Pulwama terrorist attack in Jammu and Kashmir in 2019 that killed 40 CRPF personnel and brought the two countries to the brink of a war. Modi, who was speaking on the occasion of Sardar Vallabhbhai Patel's birth anniversary, in Gujarat, said that the Pulwama attack was used by a few to promote their own selfish political interests and that the truth of the incident revealed in the Pakistan parliament has exposed them. Though the PM did not specifically name Pakistan and only said that 'claims made in Parliament of a neighbouring country' but the reference was obvious. "The country can never forget that som

PARIS: A third person was taken into custody in France in connection with the knife attack which left three dead in Nice on Thursday, a police source said on Saturday. The arrest took place on Friday, the source added. An attacker beheaded a woman and killed two other people in a church in the city of Nice on Thursday in the second deadly knife attack in France in two weeks. A man was shot by police and is now in critical condition in a hospital. France's chief anti-terrorism prosecutor has said the man suspected of carrying out the Nice attack was a Tunisian born in 1999 who had arrived in Europe on Sept. 20 in Lampedusa, the Italian island off Tunisia. On Thursday evening a 47-year-old man was taken into custody on suspicion of having been in contact with the perpetrator of the attack, a judicial source said at the time. On Friday, a 35-year old man, who is a Nice resident, and who is suspected of having met with the assailant the day before the attack, was also taken into custody, a

In [79]:
news2 = np.array(all_news)
news2 = cv.transform(news2)
# nv.predict(news2)

array([1, 0, 0, 0, 0, 0], dtype=int64)

In [80]:
logreg.predict(news2)

array([1, 0, 0, 0, 1, 0], dtype=int64)

In [ ]:
# for times of india out of 6, positive are 2 and 4 are negatives.

third channel : indian express

In [81]:
res = requests.get('https://indianexpress.com/')
soup = BeautifulSoup(res.text, 'lxml')

news_box = soup.find('div', {'class': 'lead-stories event-track-class'})
# all_news = news_box.find_all('a')
# print(all_news)
# print(news_box)
# for article in news_box.find_all('div',{'class' :'other-articles'}):
#     print(article)

links = list()
for a in news_box.find_all('a',href=True):
    if a['href'] not in links:
        links.append(a['href'])
    
for link in links:
    print( "Found the URL:",link)
    print()


all_news = list()
for link in links:
    if link != 'https://indianexpress.com/section/india/':
        res = requests.get(link)
        soup = BeautifulSoup(res.text, 'lxml')
        news_box = soup.find('div', {'class': 'full-details'})
    #     print(news_box)
        news_description = news_box.find_all('p')
        story = ""
        for news in news_description:
            story = story + news.text
        all_news.append(story)
#         print(story)
    
    
for news in all_news:
    print(news)
    print()
#     break    
    

Found the URL: https://indianexpress.com/article/india/after-punjab-now-rajasthan-introduces-three-bills-to-negate-impact-of-centres-farm-laws-6911249/

Found the URL: https://indianexpress.com/section/india/

Found the URL: https://indianexpress.com/article/india/pm-modi-pulwama-attack-pakistan-gujarat-vallabhbhai-patel-6910985/

Found the URL: https://indianexpress.com/article/sports/ipl/ipl-2020-dc-vs-mi-live-score-streaming-delhi-capitals-vs-mumbai-indians-6911238/

Found the URL: https://indianexpress.com/article/india/govt-microcredit-scheme-street-vendor-database-6910754/

Found the URL: https://indianexpress.com/article/business/future-amazon-reliance-escrow-account-6910747/

Found the URL: https://indianexpress.com/article/explained/explained-why-is-there-a-buzz-that-after-khadse-pankaja-munde-may-quit-bjp-in-maharashtra-6910304/

Following in the footsteps of Punjab, the Congress government in Rajasthan introduced three bills in the state Assembly to counter the Centre’s new 

In [82]:
news3 = np.array(all_news)
news3 = cv.transform(news3)
# nv.predict(news3)

array([0, 0, 0, 0, 1, 0], dtype=int64)

In [83]:
logreg.predict(news3)

array([0, 0, 0, 0, 1, 0], dtype=int64)

In [ ]:
#for indian express out of 6, postive is 1 and negative are 5.